# 2021/10/19
The jupypter notebook is used to test the pipleine for the ENN + SMOTE + random forest pipeline intergration from sklearn for single parameter


In [5]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [6]:
import pandas as pd
import numpy as np
import sys, os

# Pre-precess data to get the merged one

## Explore feature

In [7]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'
#df=pd.read_csv(os.path.join(input_path, 'total.Megalodon.per_read.prob.bed.gz'),compression='gzip', sep='\t')
df=pd.read_csv(os.path.join(input_path, 'total.test.bed'), sep='\t')
print("Data is loading!")

#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

df

Data is loading!


,chr,start,read_id,strand,5hmC_prob,5mC_prob,5C_prob,label
0,KJ477685.1,204,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.435397,0.085058,0.479545,2
1,KJ477685.1,261,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.342189,0.500821,0.156990,2
2,KJ477685.1,261,fe8be0cc-f4c4-49f1-80db-c840a6c7dfe9,-,0.577034,0.057676,0.365290,2
3,KJ477685.1,266,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.065316,0.861569,0.073115,2
4,KJ477685.1,266,fe8be0cc-f4c4-49f1-80db-c840a6c7dfe9,-,0.794235,0.028885,0.176880,2
...,...,...,...,...,...,...,...,...
2492,J02459.1,60,bb826a6c-c88b-45ac-9984-8b9d6bfb2f11,-,0.054875,0.013023,0.932103,0
2493,J02459.1,60,bb91267a-6661-4248-a084-554f231398c1,-,0.030720,0.004392,0.964888,0
2494,J02459.1,60,bb9cee29-89d0-4a68-a4b4-8fe5c000289f,-,0.032703,0.006934,0.960363,0
2495,J02459.1,60,bbee6b4a-acac-4db9-bbb8-2f379918b146,-,0.035734,0.003182,0.961085,0


In [8]:
df['label'].value_counts()

1    999
0    999
2    499
Name: label, dtype: int64

# Correct Strategy
We should split into training and validation folds. Then, on each fold, we should

1. Oversample the minority class with SMOTE, then undersamping with ENN

1. Train the classifier on the training fold

1. Validate the classifier on the remaining fold

https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html

https://towardsdatascience.com/the-right-way-of-using-smote-with-cross-validation-92a8d09d00c7

https://machinelearningmastery.com/combine-oversampling-and-undersampling-for-imbalanced-classification/

In [9]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, GridSearchCV
#from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [10]:
#Load the feature
#df=pd.read_csv(os.path.join(input_path, 'total.Megalodon.per_read.prob.bed.gz'),compression='gzip', sep='\t')
df=pd.read_csv(os.path.join(input_path, 'total.test.bed'), sep='\t')

#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

X = df_feature
y = df_class

In [11]:
#Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [12]:
from collections import Counter
print(Counter(y_train))

Counter({0: 799, 1: 799, 2: 399})


## Resampling strategy

In [30]:
## Define resampling strategy
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import EditedNearestNeighbours, RandomUnderSampler

In [34]:
# Define cross-validation fold
stratified_kfold = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)

In [78]:
# Make the resampling strategy: RandomUnderSampler -> SMOTE
sampler = make_pipeline(
    RandomUnderSampler(random_state = 42, sampling_strategy={0: 500, 1:500, 2:200}),
    SMOTE(sampling_strategy='not majority',
          k_neighbors=100,
          random_state = 42,
          n_jobs = -1))

X_res, y_res = sampler.fit_resample(X_train, y_train)

print(Counter(y_res))

Counter({0: 500, 1: 500, 2: 500})


In [130]:
# Make the resampling strategy: ENN -> SMOTE
ENN_SMOTE_sampler = make_pipeline(
    EditedNearestNeighbours(n_neighbors = 100,
                            n_jobs = -1),
    SMOTE(sampling_strategy='not majority',
          k_neighbors=100,
          random_state = 42,
          n_jobs = -1))

X_res_train, y_res_train = ENN_SMOTE_sampler.fit_resample(X_train, y_train)

print(Counter(y_res_train))

Counter({0: 450, 1: 450, 2: 450})


## Test with single parameter

I used the best parameter from 2021/10/13 for a test

{'rfr__bootstrap': False, 'rfr__max_depth': 25, 'rfr__max_features': 'auto', 'rfr__min_samples_leaf': 2, 'rfr__min_samples_split': 2, 'rfr__n_estimators': 120}  

In [101]:
from imblearn.pipeline import Pipeline as imbpipeline
##define random forest model
rf_model = RandomForestClassifier(
    n_estimators=10,
    max_depth=25,
    max_features = 'auto',
    min_samples_split=2,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1)

In [114]:
rf_result = rf_model.fit(X_res_train, y_res_train)
y_true, y_pred = y_test, rf_model.predict(X_test)

In [128]:
from plot_confusion_matrix import plot_cm

ModuleNotFoundError: No module named 'plot_confusion_matrix'